In [2]:
! pip install pandas

     |████████████████████████████████| 17.3 MB 7.8 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

torch 1.6.0 requires future, which is not installed.


In [4]:
! pip install numpy

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import json
import pickle
import torch.nn as nn
import torch
import torchvision.transforms as transforms
import torchvision
import easydict
import config
import random
import re
from PIL import Image

/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

cuda:1


In [4]:
category_names = {'C1':'Modality', 'C2':'Plane','C3':'Organ','C4':'Abnormality','All': 'All dataset'}

In [5]:
opt = easydict.EasyDict({
        "SEED":97,
        "BATCH_SIZE": 64,
        "VAL_BATCH_SIZE": 64,
        # "NUM_OUTPUT_UNITS": l,
        "MAX_QUESTION_LEN": 17,
        "IMAGE_CHANNEL": 1472,
        "INIT_LERARNING_RATE": 1e-4,
        "LAMNDA":0.0001,
        "MFB_FACTOR_NUM":5,
        "MFB_OUT_DIM":1000,
        "BERT_UNIT_NUM":768,
        "BERT_DROPOUT_RATIO":0.3,
        "MFB_DROPOUT_RATIO":0.1,
        "NUM_IMG_GLIMPSE":2,
        "NUM_QUESTION_GLIMPSE":2,
        "IMG_FEAT_SIZE":1,
        "IMG_INPUT_SIZE":224,
        "NUM_EPOCHS":10,
    })

In [6]:
path_change = config.path
seed_value = opt.SEED
print(seed_value) # 97
np.random.seed(seed_value) # return None
random.seed(seed_value) # return None
torch.manual_seed(seed_value) # return <torch._C.Generator object at 0x7f71cdf7a3d0>
torch.cuda.manual_seed(seed_value) # return None
torch.cuda.manual_seed_all(seed_value) # return None
torch.backends.cudnn.enabled = False 
''' backends.cudnn.enabled enables cudnn for some operations such as conv layers and RNNs, which can yield 
a significant speedup. The cudnn RNN implementation doesn’t support the backward operation during eval() 
and thus raises the error. You could disable cudnn for your workload (as already done) or try to call .train()
on the RNN module separately after using model.eval().'''
torch.backends.cudnn.benchmark = False
'''If your model does not change and your input sizes remain the same - then you may benefit from setting 
torch.backends.cudnn.benchmark = True.However, if your model changes: for instance, if you have layers that
are only "activated" when certain conditions are met, or you have layers inside a loop that can be iterated a 
different number of times, then setting torch.backends.cudnn.benchmark = True might stall your execution.'''
torch.backends.cudnn.deterministic = False
'''torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. 
Therefore, no, it will not guarantee that your training process is deterministic, since you're also using
torch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA.'''


97


"torch.backends.cudnn.deterministic=True only applies to CUDA convolution operations, and nothing else. \nTherefore, no, it will not guarantee that your training process is deterministic, since you're also using\ntorch.nn.MaxPool3d, whose backward function is nondeterministic for CUDA."

In [7]:
train_path = path_change+'/VQA_Med_2019_Dataset/Train/ImageClef-2019-VQA-Med-Training/' #QAPairsByCategory/'
train_images_path = path_change+'/VQA_Med_2019_Dataset/Train/ImageClef-2019-VQA-Med-Training/Train_images/'

valid_path = path_change+'/VQA_Med_2019_Dataset/Valid/ImageClef-2019-VQA-Med-Validation/'#QAPairsByCategory/'
valid_images_path = path_change+'/VQA_Med_2019_Dataset/Valid/ImageClef-2019-VQA-Med-Validation/Val_images/'

test_path = path_change+'/VQA_Med_2019_Dataset/Test/VQAMed2019Test/' 
test_images_path = path_change+'/VQA_Med_2019_Dataset/Test/VQAMed2019Test/VQAMed2019_Test_Images/'

In [8]:
size = opt.IMG_INPUT_SIZE
size

224

In [9]:
#Extract image feature
class VGG19(nn.Module):
    def __init__(self):
        '''
             We remove all the fully-connected layers in the VGG19 network and the convolution outputs of different feature scales
                are concatenated after global average pooling and l2-norm to form a 1984-dimensional vector to represent the image
        '''
        super(VGG19,self).__init__()
        vgg_model = torchvision.models.vgg19(pretrained=True)	

        self.Conv1 = nn.Sequential(*list(vgg_model.features.children())[0:4])
        self.Conv2 = nn.Sequential(*list(vgg_model.features.children())[4:9])
        self.Conv3 = nn.Sequential(*list(vgg_model.features.children())[9:16])
        self.Conv4 = nn.Sequential(*list(vgg_model.features.children())[16:23])
        self.Conv5 = nn.Sequential(*list(vgg_model.features.children())[23:30])
        self.Conv6 = nn.Sequential(*list(vgg_model.features.children())[30:36])

        self.avgpool = nn.Sequential(list(vgg_model.children())[1])
    
    def forward(self,image):

        with torch.no_grad():
            print('image',image.shape)
            out1 = self.Conv1(image)
            print('out1',out1.shape)
            out2 = self.Conv2(out1)
            print('out2',out2.shape)
            out3 = self.Conv3(out2)
            print('out3',out3.shape)
            out4 = self.Conv4(out3)
            print('out4',out4.shape)          
            out5 = self.Conv5(out4)          # [N, 512, 14, 14]
            print('out5',out5.shape)
            out6 = self.Conv6(out5) 
            print('out6',out6.shape)
            out7 = self.avgpool(out6)
            print('out7',out7.shape)

        #global average pooling
        out1 = out1.mean([2,3],keepdim=True)
        out2 = out2.mean([2,3],keepdim=True)
        out3 = out3.mean([2,3],keepdim=True)
        out4 = out4.mean([2,3],keepdim=True)
        out5 = out5.mean([2,3],keepdim=True)
        out6 = out6.mean([2,3],keepdim=True)
        out7 = out7.mean([2,3],keepdim=True)

       
        concat_features = torch.cat([out1, out2, out3, out4,out5,out6], 1) 
        
        #l2-normalized feature vector
        l2_norm = concat_features.norm(p=2, dim=1, keepdim=True).detach() 
        concat_features = concat_features.div(l2_norm)               # l2-normalized feature vector
        print('concat_feature',concat_features.shape)

        batch_size = concat_features.shape[0]
        embedding_dim_size = concat_features.shape[1]
        image_feature = concat_features.view(batch_size, embedding_dim_size, -1).squeeze(0) # [N, 1984, 1]


        return image_feature

In [10]:
transform = {
    phase: transforms.Compose([transforms.RandomResizedCrop(size),
                            transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.485,0.456,0.406),
                                                    (0.229,0.224,0.225))
    
    ])
    for phase in ['train','valid']
}

# transforms.RandomResizedCrop(224):
'''  This will extract a patch of size (224, 224) from your input image randomly.
So, it might pick this path from topleft, bottomright or anywhere in between. So, you are doing data augmentation 
in this part. Also, changing this value won't play nice with the fully-connected layers in your model,
so not advised to change this. '''

# transforms.RandomHorizontalFlip():
''' Once we have our image of size (224, 224), we can choose to flip it. 
This is another part of data augmentation. '''

# transforms.ToTensor():
''' This just converts your input image to PyTorch tensor. '''

# transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]):
'''  This is just input data scaling and these values
(mean and std) must have been precomputed for your dataset. Changing these values is also not advised. '''


test_transform = transforms.Compose([transforms.RandomResizedCrop(size),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.485,0.456,0.406),
                                                            (0.229,0.224,0.225))])
transform
test_transform

Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
    ToTensor()
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)

In [11]:
def mixup_data_vqa(v, q, a, alpha=1.0, use_cuda=True):
    '''Returns mixed images, pairs of questions, targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = v.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_v = lam * v + (1 - lam) * v[index, :]
    a_a, a_b = a, a[index]
    q_a, q_b = q, q[index]
    return mixed_v, a_a, a_b, q_a, q_b, lam


In [12]:
def count_answer_freq(df_data):
    '''
    count the frequence of each unique answer on the dataset
    '''
    all_answers = df_data['Answer'].values
    answer_freq_dict = defaultdict(int)
    for answer in all_answers:
        answer_freq_dict[answer] += 1
    answer_freq_dict_sort = dict(sorted(answer_freq_dict.items(), key=lambda x: x[1], reverse=True))

    return answer_freq_dict_sort

In [13]:
def get_most_frequent_classes(answer_freq_dict, threshold=1):
    print('threshold',threshold)
    final_classes = defaultdict(int)
    index = 0

    for answer,ans_freq in answer_freq_dict.items():

        if ans_freq >= threshold:
            final_classes[answer]= index         
        else:
            final_classes['UNKNOWN'] = index
            break
        index += 1  
    final_classes['UNKNOWN'] = index
    with open('answer_classes.json', 'w') as fp:
        json.dump(final_classes, fp)
    return final_classes

In [14]:
def get_category_file_train(category_name,category_path,images_path,vgg19_model,transform=None):
    data = []

    dict_data = {
        'Image_id':[],
        'Question': [],
        'Answer':[],
        'image':[]
    }

    image_feat={}

    with open(category_path) as f:
        lines = f.readlines()

    cnt=0
    for element in lines:
        pd_element = element.split('|')
        dict_data['Image_id'].append(pd_element[0])
        dict_data['Question'].append(re.sub(r'\s+', ' ',pd_element[1]).strip())
        dict_data['Answer'].append(pd_element[2].strip("\n"))

        image = Image.open(images_path+pd_element[0]+'.jpg').convert('RGB')
        if transform:
            image = transform(image)
        print('image.shape',image.shape)
        print('shape',image[None,...].shape)
        dict_data['image'].append(image[None,...])
        # image_feature = vgg19_model(image[None,...].to(device))

        # print('iamge_feature',image_feature.shape)
        # image_feat[pd_element[0]] = image_feature.cpu().numpy()
        cnt=cnt+1
    

    df_data = pd.DataFrame(dict_data,columns = ['Image_id', 'image','Question','Answer'])
    
    answer_freq = count_answer_freq(df_data)

    classes = get_most_frequent_classes(answer_freq, threshold=5)

    df_data['labels'] = df_data['Answer'].apply(lambda x : classes[x] if x in classes else classes['UNKNOWN'])
    
    # print(dict_data)
    # print(cnt)

    return df_data,classes,image_feat

In [15]:
 #read a txt validation file and structure it in a dataframe
def get_category_file_valid(category_name, category_path ,images_path, classes, vgg19_model, transform=None):
    
    data = []
    
    dict_data = { 'Image_id':[],'image':[],'Question':[],'Answer':[] }
    image_feat = { }
    with open(category_path) as f:
        lines = f.readlines()
    
    for element in lines:
        pd_element = element.split('|')
        dict_data['Image_id'].append(pd_element[0])
        dict_data['Question'].append(re.sub(r'\s+', ' ',pd_element[1]).strip())
        dict_data['Answer'].append(pd_element[2].strip("\n")) 
        image = Image.open(images_path+pd_element[0]+'.jpg').convert('RGB')
        if transform:
            image = transform(image)
        dict_data['image'].append(image[None,...])
        image_feature = vgg19_model(image[None,...].to(device))
        image_feat[pd_element[0]] = image_feature.cpu().numpy()
        
    df_data = pd.DataFrame(dict_data, columns = ['Image_id',
                                                    'image', 
                                                 'Question',
                                                 'Answer',
                                                 ])
    
    df_data['labels'] = df_data['Answer'].apply(lambda x : classes[x] if x in classes else classes['UNKNOWN'])

    
    return df_data ,image_feat


In [16]:
#read a txt test file and structure it in a dataframe
def get_test_file(category_name, category_path ,images_path,vgg19_model, transform=None,group='test'):
    
    data = []
    
    dict_data = { 'Image_id':[],
                    'image':[],
                'Question':[],
                'Group':[],
              }

    image_feat = { }
    with open(category_path) as f:
        lines = f.readlines()
    
    for element in lines:
        pd_element = element.split('|')
        dict_data['Image_id'].append(pd_element[0])
        dict_data['Question'].append(re.sub(r'\s+', ' ',pd_element[1]).strip("\n")) 
        dict_data['Group'].append(group)
        image = Image.open(images_path+pd_element[0]+'.jpg').convert('RGB')
        if transform:
            image = transform(image)
        dict_data['image'].append(image[None,...])
        image_feature = vgg19_model(image[None,...].to(device))
        image_feat[pd_element[0]] = image_feature.cpu().numpy()
        
    df_data = pd.DataFrame(dict_data, columns = ['Image_id',
                                                    'image',
                                                 'Question',
                                                 'Group', 
                                                 ])
    return df_data,image_feat

In [17]:
vgg19_model = VGG19().to(device)

In [18]:
train_dataset_df, classes, image_feat_train = get_category_file_train(category_name=category_names['All'],
                                                        category_path=train_path+'All_QA_Pairs_train.txt', 
                                                        images_path=train_images_path,
                                                        vgg19_model=vgg19_model,
                                                        transform=transform['train'])

image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.Size([1, 3, 224, 224])
image.shape torch.Size([3, 224, 224])
shape torch.S

In [23]:
train_dataset_df

Image_id                                              image  \
0      synpic41148  [[[tensor([-1.3815, -1.3815, -1.4500, -1.4500,...   
1      synpic43984  [[[tensor([-2.1008, -2.1008, -2.1008, -2.1008,...   
2      synpic38930  [[[tensor([-2.1179, -2.1179, -2.1179, -2.1179,...   
3      synpic52143  [[[tensor([-1.7754, -1.7583, -1.7583, -1.7754,...   
4      synpic20934  [[[tensor([-2.1008, -2.1008, -2.1008, -2.1008,...   
...            ...                                                ...   
12787  synpic35685  [[[tensor([-1.5699, -1.5699, -1.5699, -1.5699,...   
12788  synpic35687  [[[tensor([-1.5699, -1.5699, -1.5699, -1.5699,...   
12789  synpic35701  [[[tensor([-1.5699, -1.5699, -1.5699, -1.5699,...   
12790  synpic48036  [[[tensor([-2.1179, -2.1179, -2.1179, -2.1179,...   
12791  synpic54897  [[[tensor([ 0.2967,  0.2967,  0.2967,  0.2967,...   

                                                Question  \
0                            what kind of image is this?   
1                           is this a t1 weighted image?   
2      what type of imaging modality is used to acqui...   
3                             is this a noncontrast mri?   
4                   what type of image modality is this?   
...                                                  ...   
12787                              is this image normal?   
12788             is there something wrong in the image?   
12789            is there evidence of any abnormalities?   
12790                             is the ct scan normal?   
12791                is there an abnormality in the mri?   

                     Answer  labels  
0      cta - ct angiography      32  
1                        no       2  
2           us - ultrasound      15  
3                        no       2  
4           xr - plain film       5  
...                     ...     ...  
12787                   yes       3  
12788                    no       2  
12789                    no       2  
12790                   yes       3  
12791                    no       2  

[12792 rows x 5 columns]

In [24]:
valid_dataset_df,image_feat_valid = get_category_file_valid(category_name=category_names['All'],
                                                            category_path=valid_path + 'All_QA_Pairs_val.txt',
                                                            images_path=valid_images_path,
                                                            classes=classes,
                                                            vgg19_model=vgg19_model,
                                                            transform=transform['valid'])

image torch.Size([1, 3, 224, 224])
out1 torch.Size([1, 64, 224, 224])
out2 torch.Size([1, 128, 112, 112])
out3 torch.Size([1, 256, 56, 56])
out4 torch.Size([1, 512, 28, 28])
out5 torch.Size([1, 512, 14, 14])
out6 torch.Size([1, 512, 14, 14])
out7 torch.Size([1, 512, 7, 7])
concat_feature torch.Size([1, 1984, 1, 1])
image torch.Size([1, 3, 224, 224])
out1 torch.Size([1, 64, 224, 224])
out2 torch.Size([1, 128, 112, 112])
out3 torch.Size([1, 256, 56, 56])
out4 torch.Size([1, 512, 28, 28])
out5 torch.Size([1, 512, 14, 14])
out6 torch.Size([1, 512, 14, 14])
out7 torch.Size([1, 512, 7, 7])
concat_feature torch.Size([1, 1984, 1, 1])
image torch.Size([1, 3, 224, 224])
out1 torch.Size([1, 64, 224, 224])
out2 torch.Size([1, 128, 112, 112])
out3 torch.Size([1, 256, 56, 56])
out4 torch.Size([1, 512, 28, 28])
out5 torch.Size([1, 512, 14, 14])
out6 torch.Size([1, 512, 14, 14])
out7 torch.Size([1, 512, 7, 7])
concat_feature torch.Size([1, 1984, 1, 1])
image torch.Size([1, 3, 224, 224])
out1 torch.Siz

In [25]:
valid_dataset_df

Image_id                                              image  \
0     synpic54733  [[[tensor([-2.1179, -2.1179, -2.1179, -2.1179,...   
1     synpic25647  [[[tensor([-0.7650, -0.7993, -0.7650, -0.7308,...   
2     synpic35681  [[[tensor([-1.0219, -1.0219, -1.0219, -1.0048,...   
3     synpic39641  [[[tensor([-2.1179, -2.1179, -2.1179, -2.1179,...   
4     synpic35693  [[[tensor([-0.8849, -0.9877, -1.1418, -1.2617,...   
...           ...                                                ...   
1995  synpic45828  [[[tensor([-2.0323, -2.0323, -2.0323, -2.0323,...   
1996  synpic60458  [[[tensor([-1.5528, -1.5357, -1.5014, -1.4500,...   
1997  synpic43524  [[[tensor([-2.1179, -2.1179, -2.1179, -2.1179,...   
1998  synpic59568  [[[tensor([-1.8268, -1.8097, -1.8097, -1.8097,...   
1999  synpic48459  [[[tensor([-1.2788, -1.3302, -1.4329, -1.5528,...   

                                               Question  \
0     what imaging modality was used to take this im...   
1                            what kind of scan is this?   
2                      was the mri taken with contrast?   
3     is this a t1 weighted, t2 weighted, or flair i...   
4                            is this a noncontrast mri?   
...                                                 ...   
1995                       what is abnormal in the mri?   
1996                     what is abnormal in the x-ray?   
1997              what is most alarming about this mri?   
1998                       what is abnormal in the mri?   
1999              what is most alarming about this mri?   

                                               Answer  labels  
0                                 ct with iv contrast      33  
1                                     xr - plain film       5  
2                                                  no       2  
3                                                  t2      12  
4                                                 yes       3  
...                                               ...     ...  
1995  marked hydrocephalus due to aqueductal stenosis     177  
1996                                 discoid meniscus     177  
1997    choroid plexus neoplasm, papilloma, carcinoma     127  
1998                               multiple sclerosis      62  
1999                           spine,  tethered  cord     177  

[2000 rows x 5 columns]

In [26]:
test_dataset_df, image_feat_test = get_test_file(category_name=category_names['All'], 
                                                  category_path=test_path+'VQAMed2019_Test_Questions.txt'  ,
                                                  images_path=test_images_path,
                                                  vgg19_model=vgg19_model,
                                                  transform=test_transform,
                                                  group='test')


image torch.Size([1, 3, 224, 224])
out1 torch.Size([1, 64, 224, 224])
out2 torch.Size([1, 128, 112, 112])
out3 torch.Size([1, 256, 56, 56])
out4 torch.Size([1, 512, 28, 28])
out5 torch.Size([1, 512, 14, 14])
out6 torch.Size([1, 512, 14, 14])
out7 torch.Size([1, 512, 7, 7])
concat_feature torch.Size([1, 1984, 1, 1])
image torch.Size([1, 3, 224, 224])
out1 torch.Size([1, 64, 224, 224])
out2 torch.Size([1, 128, 112, 112])
out3 torch.Size([1, 256, 56, 56])
out4 torch.Size([1, 512, 28, 28])
out5 torch.Size([1, 512, 14, 14])
out6 torch.Size([1, 512, 14, 14])
out7 torch.Size([1, 512, 7, 7])
concat_feature torch.Size([1, 1984, 1, 1])
image torch.Size([1, 3, 224, 224])
out1 torch.Size([1, 64, 224, 224])
out2 torch.Size([1, 128, 112, 112])
out3 torch.Size([1, 256, 56, 56])
out4 torch.Size([1, 512, 28, 28])
out5 torch.Size([1, 512, 14, 14])
out6 torch.Size([1, 512, 14, 14])
out7 torch.Size([1, 512, 7, 7])
concat_feature torch.Size([1, 1984, 1, 1])
image torch.Size([1, 3, 224, 224])
out1 torch.Siz

In [27]:
test_dataset_df

Image_id                                              image  \
0    synpic54082  [[[tensor([-0.3712, -0.3027, -0.2856, -0.2684,...   
1    synpic48556  [[[tensor([-2.1008, -2.1008, -2.1008, -2.1008,...   
2    synpic50696  [[[tensor([-0.3712, -0.3712, -0.3541, -0.3198,...   
3    synpic37194  [[[tensor([-0.9534, -0.7650, -0.4568, -0.3027,...   
4    synpic31308  [[[tensor([-1.6555, -1.6213, -1.5870, -1.5528,...   
..           ...                                                ...   
495  synpic21789  [[[tensor([-0.5082, -0.5082, -0.4911, -0.4911,...   
496  synpic39878  [[[tensor([-1.8953, -1.9124, -1.8953, -1.8782,...   
497  synpic41525  [[[tensor([-2.1008, -2.1008, -2.1008, -2.1008,...   
498  synpic18173  [[[tensor([-0.6965, -0.6623, -0.6452, -0.7993,...   
499  synpic54143  [[[tensor([-1.0733, -1.1418, -1.1247, -1.1075,...   

                                            Question Group  
0                           what modality is shown?   test  
1                  was the mri taken with contrast?   test  
2      what type of contrast did this patient have?   test  
3                     what imaging method was used?   test  
4                           what modality is shown?   test  
..                                               ...   ...  
495                what is abnormal in the ct scan?   test  
496           what is most alarming about this mri?   test  
497       what is most alarming about this ct scan?   test  
498  what is the primary abnormality in this image?   test  
499  what is the primary abnormality in this image?   test  

[500 rows x 4 columns]

In [28]:
#save preprocessed training data frame
def save_preprocesse_data(dataset_df, pickle_name, image_feat, name= 'train'):
    path_output_chd = config.path_output_chd
    if name == 'train':
        
        dataset_df.to_pickle(path_output_chd+'/train_dataset_pickle/'+pickle_name+'.pkl')
        with open(path_output_chd+'/train_dataset_pickle/train-image-feature.pickle', 'wb') as f:

            # Pickle the 'data' dictionary using the highest protocol available.
            pickle.dump(image_feat, f, pickle.HIGHEST_PROTOCOL)
        
    elif name == 'valid':
        
        dataset_df.to_pickle(path_output_chd+'/valid_dataset_pickle/'+pickle_name+'.pkl')

        with open(path_output_chd+'/valid_dataset_pickle/valid-image-feature.pickle', 'wb') as f:

            # Pickle the 'data' dictionary using the highest protocol available.
            pickle.dump(image_feat, f, pickle.HIGHEST_PROTOCOL)
        
    else: 
       
        dataset_df.to_pickle(path_output_chd+'/test_dataset_pickle/'+pickle_name+'.pkl')
        with open(path_output_chd+'/test_dataset_pickle/test-image-feature.pickle', 'wb') as f:

            # Pickle the 'data' dictionary using the highest protocol available.
            pickle.dump(image_feat, f, pickle.HIGHEST_PROTOCOL)




In [ ]:
#save preprocessed training data frame
def save_preprocesse_data(dataset_df, pickle_name, image_feat, name= 'train'):
    path_output_chd = config.path_output_chd
    if name == 'train':
        
        dataset_df.to_pickle(path_output_chd+'/train_dataset_pickle/'+pickle_name+'.pkl')
        with open(path_output_chd+'/train_dataset_pickle/train-image-feature.pickle', 'wb') as f:

            # Pickle the 'data' dictionary using the highest protocol available.
            pickle.dump(image_feat, f, pickle.HIGHEST_PROTOCOL)
        
    elif name == 'valid':
        
        dataset_df.to_pickle(path_output_chd+'/valid_dataset_pickle/'+pickle_name+'.pkl')

        with open(path_output_chd+'/valid_dataset_pickle/valid-image-feature.pickle', 'wb') as f:

            # Pickle the 'data' dictionary using the highest protocol available.
            pickle.dump(image_feat, f, pickle.HIGHEST_PROTOCOL)
        
    else: 
       
        dataset_df.to_pickle(path_output_chd+'/test_dataset_pickle/'+pickle_name+'.pkl')
        with open(path_output_chd+'/test_dataset_pickle/test-image-feature.pickle', 'wb') as f:

            # Pickle the 'data' dictionary using the highest protocol available.
            pickle.dump(image_feat, f, pickle.HIGHEST_PROTOCOL)



In [ ]:
####################################################SAVE AND LOAD DATA FRAME########################################################
save_preprocesse_data(train_dataset_df, 'train_dataset_df', image_feat_train,'train')
save_preprocesse_data(valid_dataset_df, 'valid_dataset_df', image_feat_valid,'valid')
save_preprocesse_data(test_dataset_df, 'C1_test_dataset_df',image_feat_test, 'test')